# Suspicious Keyboard Layout Load
Detects the keyboard preload installation with a suspicious keyboard layout, e.g. Chinese, Iranian or Vietnamese layout load in user session on systems maintained by US staff only

## Rule Content
```
- title: Suspicious Keyboard Layout Load
  id: 34aa0252-6039-40ff-951f-939fd6ce47d8
  description: Detects the keyboard preload installation with a suspicious keyboard
    layout, e.g. Chinese, Iranian or Vietnamese layout load in user session on systems
    maintained by US staff only
  references:
  - https://renenyffenegger.ch/notes/Windows/registry/tree/HKEY_CURRENT_USER/Keyboard-Layout/Preload/index
  - https://github.com/SwiftOnSecurity/sysmon-config/pull/92/files
  author: Florian Roth
  date: 2019/10/12
  modified: 2019/10/15
  logsource:
    product: windows
    service: sysmon
    definition: 'Requirements: Sysmon config that monitors \Keyboard Layout\Preload
      subkey of the HKLU hives - see https://github.com/SwiftOnSecurity/sysmon-config/pull/92/files'
    category: null
  detection:
    selection_registry:
      EventID: 13
      TargetObject:
      - '*\Keyboard Layout\Preload\*'
      - '*\Keyboard Layout\Substitutes\*'
      Details:
      - 00000429
      - 00050429
      - 0000042a
    condition: selection_registry
  falsepositives:
  - Administrators or users that actually use the selected keyboard layouts (heavily
    depends on the organisation's user base)
  level: medium

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-sysmon-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:"13" AND registry_key_path.keyword:(*\\Keyboard\ Layout\\Preload\* OR *\\Keyboard\ Layout\\Substitutes\*) AND registry_key_value:("00000429" OR "00050429" OR "0000042a"))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()